## Setup & Imports

In [1]:
from pathlib import Path
import sys

import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sys.path.append("../")

from src.readers.geom_reader import load_geodata
from src.readers.results_reader import read_conceptual_results
from src.timeseries_stats.bias_analysis import (
    calculate_flow_duration_bias,
    calculate_monthly_bias,
    calculate_seasonal_bias,
    classify_bias_magnitude,
)
from src.timeseries_stats.metrics import calculate_bias, calculate_nse
from src.utils.logger import setup_logger

plt.rcParams["font.family"] = "DeJavu Serif"
plt.rcParams["font.serif"] = ["Times New Roman"]

log = setup_logger("chapter_three", log_file="../logs/chapter_three.log")

ModuleNotFoundError: No module named 'src.readers.results_reader'

## 1. Load Data

In [ ]:
# Load watershed geometries and gauge locations
ws, gauges = load_geodata(folder_depth="../")
common_index = gauges.index.to_list()

# Load cluster assignments (from Chapter 1)
gauge_mapping = pd.read_csv(
    "../res/chapter_one/gauge_hybrid_mapping.csv",
    index_col="gauge_id",
    dtype={"gauge_id": str},
)

# Load HydroATLAS attributes
geo_data = pd.read_csv(
    "../data/attributes/hydro_atlas_cis_camels.csv",
    index_col="gauge_id",
    dtype={"gauge_id": str},
)

print(f"Loaded {len(gauges)} gauges with attributes")

In [ ]:
# Load model results (using GR4J as example)
gr4j_dir = Path("../test/gr4j")
model_results = read_conceptual_results(
    gr4j_dir, model_name="GR4J", common_index=common_index
)

print(f"Loaded results for {len(model_results)} gauges")

## 2. Overall Bias Analysis

In [ ]:
# Calculate overall bias for each gauge
bias_data = []

for gauge_id, data in model_results.items():
    obs = data["obs"]
    sim = data["sim"]

    # Overall bias (% error)
    bias_pct = calculate_bias(obs, sim)
    nse = calculate_nse(obs, sim)

    # Classify bias magnitude
    bias_class = classify_bias_magnitude(bias_pct)

    bias_data.append(
        {
            "gauge_id": gauge_id,
            "bias_pct": bias_pct,
            "nse": nse,
            "bias_class": bias_class,
        }
    )

bias_df = pd.DataFrame(bias_data).set_index("gauge_id")
print(f"\nBias statistics:")
print(f"Mean bias: {bias_df['bias_pct'].mean():.2f}%")
print(f"Median bias: {bias_df['bias_pct'].median():.2f}%")
print(f"\nBias distribution:")
print(bias_df["bias_class"].value_counts())

In [ ]:
# Bias vs NSE scatter plot
fig, ax = plt.subplots(figsize=(10, 6))

scatter = ax.scatter(
    bias_df["bias_pct"],
    bias_df["nse"],
    c=bias_df["nse"],
    cmap="RdYlGn",
    vmin=0,
    vmax=1,
    alpha=0.6,
)

ax.axvline(0, color="red", linestyle="--", alpha=0.5, label="Zero bias")
ax.axhline(0.5, color="blue", linestyle="--", alpha=0.5, label="NSE threshold")

ax.set_xlabel("Bias (%)")
ax.set_ylabel("NSE")
ax.set_title("Model Bias vs Performance")
ax.legend()
plt.colorbar(scatter, ax=ax, label="NSE")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig("../res/chapter_three/bias_vs_nse.png", dpi=300)
plt.show()

## 3. Seasonal Bias Patterns

In [ ]:
# Calculate monthly bias for all gauges
monthly_bias = {}

for gauge_id, data in model_results.items():
    obs = data["obs"]
    sim = data["sim"]

    monthly_bias[gauge_id] = calculate_monthly_bias(obs, sim)

monthly_bias_df = pd.DataFrame(monthly_bias).T
print("\nMean monthly bias (%)")
print(monthly_bias_df.mean())

In [ ]:
# Plot seasonal bias patterns
fig, ax = plt.subplots(figsize=(12, 6))

# Plot individual gauges (light)
for gauge_id in monthly_bias_df.index:
    ax.plot(range(1, 13), monthly_bias_df.loc[gauge_id], alpha=0.1, color="gray")

# Plot mean (bold)
mean_monthly_bias = monthly_bias_df.mean()
ax.plot(
    range(1, 13),
    mean_monthly_bias,
    linewidth=3,
    color="red",
    label="Mean bias",
)

ax.axhline(0, color="black", linestyle="--", alpha=0.5)
ax.fill_between(
    range(1, 13),
    monthly_bias_df.quantile(0.25),
    monthly_bias_df.quantile(0.75),
    alpha=0.3,
    color="red",
    label="IQR",
)

ax.set_xlabel("Month")
ax.set_ylabel("Bias (%)")
ax.set_title("Monthly Bias Patterns (All Gauges)")
ax.set_xticks(range(1, 13))
ax.set_xticklabels(
    ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
)
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig("../res/chapter_three/monthly_bias_patterns.png", dpi=300)
plt.show()

## 4. Flow Regime Bias (Flow Duration Curve)

In [ ]:
# Calculate FDC bias for all gauges
fdc_bias = {}

for gauge_id, data in model_results.items():
    obs = data["obs"]
    sim = data["sim"]

    fdc_bias[gauge_id] = calculate_flow_duration_bias(obs, sim)

fdc_bias_df = pd.DataFrame(fdc_bias).T
print("\nMean FDC bias (%) by exceedance probability:")
print(fdc_bias_df.mean())

In [ ]:
# Plot FDC bias patterns
fig, ax = plt.subplots(figsize=(12, 6))

# Plot individual gauges (light)
percentiles = np.arange(5, 100, 5)
for gauge_id in fdc_bias_df.index:
    ax.plot(percentiles, fdc_bias_df.loc[gauge_id], alpha=0.1, color="gray")

# Plot mean (bold)
mean_fdc_bias = fdc_bias_df.mean()
ax.plot(percentiles, mean_fdc_bias, linewidth=3, color="red", label="Mean bias")

ax.axhline(0, color="black", linestyle="--", alpha=0.5)
ax.fill_between(
    percentiles,
    fdc_bias_df.quantile(0.25),
    fdc_bias_df.quantile(0.75),
    alpha=0.3,
    color="red",
    label="IQR",
)

# Add flow regime annotations
ax.axvspan(0, 20, alpha=0.1, color="blue", label="High flows")
ax.axvspan(20, 80, alpha=0.1, color="green", label="Mid flows")
ax.axvspan(80, 100, alpha=0.1, color="orange", label="Low flows")

ax.set_xlabel("Exceedance Probability (%)")
ax.set_ylabel("Bias (%)")
ax.set_title("Flow Duration Curve Bias (All Gauges)")
ax.legend(loc="upper right")
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig("../res/chapter_three/fdc_bias_patterns.png", dpi=300)
plt.show()

## 5. Spatial Bias Distribution

In [ ]:
# Merge bias with gauge locations
gauges_with_bias = gauges.join(bias_df[["bias_pct", "bias_class"]])

# Plot map
fig, ax = plt.subplots(
    figsize=(15, 10), subplot_kw={"projection": ccrs.LambertConformal()}
)

# Add basemap
basemap_data = gpd.read_file("../data/geometry/basemap_2023.gpkg")
basemap_data.to_crs(ccrs.LambertConformal()).plot(
    ax=ax, color="lightgray", edgecolor="black", linewidth=0.5
)

# Plot gauges colored by bias
gauges_with_bias.to_crs(ccrs.LambertConformal()).plot(
    column="bias_pct",
    ax=ax,
    cmap="RdBu_r",
    vmin=-50,
    vmax=50,
    markersize=30,
    legend=True,
    legend_kwds={"label": "Bias (%)", "shrink": 0.7},
)

ax.set_title("Spatial Distribution of Model Bias")
plt.tight_layout()
plt.savefig("../res/chapter_three/spatial_bias_map.png", dpi=300)
plt.show()

## 6. Bias Attribution Analysis

In [ ]:
# Merge bias with physiographic attributes
bias_with_attrs = bias_df.join(geo_data)

# Calculate correlations
from src.constants.features import STANDARD_FEATURES

correlations = (
    bias_with_attrs[["bias_pct"] + STANDARD_FEATURES].corr()["bias_pct"].drop("bias_pct")
)
correlations = correlations.sort_values(ascending=False)

print("\nTop 5 positive correlations with bias:")
print(correlations.head())
print("\nTop 5 negative correlations with bias:")
print(correlations.tail())

In [ ]:
# Plot correlation heatmap
fig, ax = plt.subplots(figsize=(10, 8))

sns.barplot(x=correlations.values, y=correlations.index, ax=ax)
ax.axvline(0, color="red", linestyle="--", alpha=0.5)
ax.set_xlabel("Correlation with Bias")
ax.set_ylabel("Feature")
ax.set_title("Physiographic Controls on Model Bias")
plt.tight_layout()
plt.savefig("../res/chapter_three/bias_attribution.png", dpi=300)
plt.show()

## 7. Bias by Cluster

In [ ]:
# Merge with cluster assignments
bias_with_clusters = bias_df.join(gauge_mapping[["hybrid_class"]])

# Calculate mean bias by cluster
cluster_bias = bias_with_clusters.groupby("hybrid_class")["bias_pct"].agg(
    ["mean", "median", "std", "count"]
)
cluster_bias = cluster_bias.sort_values("mean")

print("\nMean bias by cluster:")
print(cluster_bias)

In [ ]:
# Box plot
fig, ax = plt.subplots(figsize=(12, 8))

sns.boxplot(data=bias_with_clusters, x="hybrid_class", y="bias_pct", ax=ax)
ax.axhline(0, color="red", linestyle="--", alpha=0.5, label="Zero bias")
ax.set_xlabel("Hybrid Class")
ax.set_ylabel("Bias (%)")
ax.set_title("Bias Distribution by Hybrid Class")
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("../res/chapter_three/bias_by_cluster.png", dpi=300)
plt.show()

## 8. Export Results

In [ ]:
# Export bias statistics
bias_df.to_csv("../res/chapter_three/gauge_bias.csv")
monthly_bias_df.to_csv("../res/chapter_three/monthly_bias.csv")
fdc_bias_df.to_csv("../res/chapter_three/fdc_bias.csv")
cluster_bias.to_csv("../res/chapter_three/cluster_bias_summary.csv")

# Export attribution analysis
correlations.to_csv("../res/chapter_three/bias_feature_correlations.csv")

print("\n✓ Exported bias analysis results to res/chapter_three/")

## Summary

- **Overall bias:** Mean bias varies across gauges, with systematic seasonal patterns
- **Seasonal patterns:** Spring snowmelt periods show distinct bias characteristics
- **Flow regime:** Low flows tend to be underestimated, high flows overestimated
- **Spatial distribution:** Bias shows geographical clustering
- **Attribution:** Elevation, forest cover, and aridity index correlate with bias magnitude
- **Cluster-specific bias:** Hybrid classes show distinct bias signatures